<img src="https://www.rochester.edu/assets/images/ur-logo.svg">

# <center>[CSC 249/449: Machine Vision](https://www.cs.rochester.edu/~cxu22/t/249S22/)</center>


1. Make sure you fill in all cells contain `YOUR CODE HERE` or `YOUR ANSWER HERE`.
2. After you finished, `Restart the kernel & run` all cell in order.
---------


# Motion Estimation

This project helps you get your hands on estimating pixel motion by solving **optical flow** problem. Given two consecutive frames with small motion, the machine should be able to estimate the motion by following **brightness constancy**.

As discussed in the class, we can transform the problem of motion estimation into solving the equation with **brightness constancy** and **spatial coherence** constraints:

$$
\nabla I(p_i) \cdot [u,v] + I_t(p_i) = 0
$$

The number of equations per pixel is related to the selected window size. 

In this project, the overall pipeline is decomposed to two parts:

1. **Compute Gradients**: You will need to get the image gradients along x, y, and also the time axes. You should be familiar with this since you have already experienced how to compute gradients in the previous problem set.
2. **Flow Estimation**: Given the image gradients, you will solve the aperture problem to obtain the $[u,v]$ for pixels. 

In [ ]:
import cv2
import csc249
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
# read current frame and the next frame
img_cur = cv2.cvtColor(cv2.imread(str(csc249.data/'basketball1.png')), cv2.COLOR_BGR2GRAY)
img_next = cv2.cvtColor(cv2.imread(str(csc249.data/'basketball2.png')), cv2.COLOR_BGR2GRAY)

fig = plt.figure(figsize=(10, 7))
fig.add_subplot(1, 2, 1)
plt.imshow(img_cur, cmap='gray')
plt.title("Current Img")
fig.add_subplot(1, 2, 2)
plt.imshow(img_next, cmap='gray')
plt.title("Next Img")

##  Compute Gradients

Recall the steps of image filtering and gradients:

1. Generate image filters along $x$, $y$, and $time$ axes, respectively.(3 points)
2. Apply the filters on images to get the gradients(7 points)
    * The filters X, Y should be applied on the current image, while the gradients should be computed from both the current and the next image
    * Use `cv2.filter2D` to implement the filtering    

In [ ]:
# return X,Y,T filters
def firstorder():
    """
    Returns:
    --------
        filter X: numpy.ndarray
        filter Y: numpy.ndarray  
        filter T: numpy.ndarray
    """
    # YOUR CODE HERE
    X = [-1, 1]
    Y = [[-1], [1]]
    T = [1]
    #raise NotImplementedError()
    return {'filterX': X,
            'filterY':Y, 
            'filterT': T}

If correctly implemented, you should get three filters that will be passed into the next function.

In [ ]:
# compute gradients on images
def get_gradient(img_cur, img_next, filters):
    """
    Arguments:
    ----------
        img_cur, img_next : the current and the next grayscale images
            A 2D numpy.ndarray of shape H x W.
        filters: filters
            a dictonary of numpy.ndarray
    Returns:
    --------
        grad_x, grad_y, grad_t: numpy.ndarray
            H x W array of gradients in image coordinates
    """
    X = filters['filterX']
    Y = filters['filterY']
    T = filters['filterT']
    # YOUR CODE HERE
    gx = cv2.filter2D(img_cur, -1, np.array(X))
    gy = cv2.filter2D(img_cur, -1, np.array(Y))
    gtcur = cv2.filter2D(img_cur, -1, np.array(T))
    gtnext = cv2.filter2D(img_next, -1, np.ndarray(T))
    gt = gtnext - gtcur
    #raise NotImplementedError()
    return {'grad_x': gx, 
            'grad_y': gy, 
            'grad_t': gt}

## Flow Estimation(15 points)

Refer to the class, solving the aperture problem can help get the pixel movement $[u,v]$. Other than estimating motion on all the pixels, we first detect a set of good feature points. In this case, you are actually iterating the flow estimation on the selected points. The function involves the following steps:

1. Unpack the [x,y] coordinates from the detected corners
    * The return coordinates are in [y,x] format
2. Compute the $I_x$, $I_y$ and $I_t$ for each neighbors inside the window
    * The $I_x$, $I_y$ and $I_t$ may be in 2D shape, you can use `.flatten()` to make them into 1D arrays
3. If you use 5x5 for the windows, now you will have 25 equations to compute the $(u,v)$ for each pixel. The overall equation is $A$$d$ = $b$ (the same as in class materials). Then you can get this $(u,v)$ by solving least squares problem
    * Hint: use `np.linalg.pinv()` function to get the matrix that solves the least-squares problem
4. After running over all the detected points, return the updated flow
    

In [ ]:
def solving_aperture(flow, corners, grads, window_size):
    """
    Arguments:
    ----------
        flow: the initial flow with all pixels of zero values
            Two 2D numpy.ndarray [u, v] of shape H x W.
        corners: the detected strong corners on image
            a list of [y, x] coordinates
        grads: the image gradients
        window_size: the number of neighbors/equations to compute [u,v] 
    Returns:
    --------
        u, v: numpy.ndarray
            two H x W arrays representing the flow
    """
    # extract gradients
    gx = grads['grad_x']
    gy = grads['grad_y']
    gt = grads['grad_t']
    
    # extract u, v
    u = flow['u']
    v = flow['v']
    
    # YOUR CODE HERE
    # We first pad the gradients to ensure that no out-of-bound error occurs.
    # We then compute u, v at each of the strong corners detected using the formula in the lecture.
    m = window_size // 2
    Gx = np.pad(gx, pad_width = m)
    Gy = np.pad(gy, pad_width = m)
    Gt = np.pad(gt, pad_width = m)
    for coor in corners:
        x = coor[1]
        y = coor[0]
        A = []
        b = []
        for i in range(window_size):
            for j in range(window_size):
                A.append([Gx[x + i][y + j], Gy[x + i][y + j]])
                b.append(-Gt[x + i][y + j])
        A = np.array(A)
        b = np.array(b)
        b = np.transpose(b)
        d = np.matmul(np.linalg.pinv(A), b)
        u[x][y] = d.item(1)
        v[x][y] = d.item(0)
    #raise NotImplementedError()
    return {'u':u,
           'v':v}

## Visualization


If you got it right so far, you should be able to get your flow visualization similar to this:

![alt text](result.png "Flow Result Example")


In [ ]:
max_corners=10000
min_quality=0.01
min_distance=0.1
corners = cv2.goodFeaturesToTrack(img_cur, max_corners, min_quality, min_distance)
corners = np.squeeze(corners).astype(int)

img_cur = img_cur / 255
img_next = img_next / 255

filters = firstorder()

grads = get_gradient(img_cur, img_next, filters)

# initialize the flow
u = np.zeros(img_cur.shape)
v = np.zeros(img_cur.shape)
flow = {'u':u, 'v':v}

# sovling aperture problem
flow = solving_aperture(flow, corners, grads, window_size=5)

# show results
img_to_show = cv2.cvtColor((img_next*255).astype('uint8'), cv2.COLOR_GRAY2RGB)
csc249.plotFlow(img_to_show, flow)

# Submission

1. At the menubar, click `Kernel`$\rightarrow$ `Restart & Run All`
2. Download the zip file and upload via blackboard
   
1% deduction of late assignment total score per hour passing the deadline.

In [ ]:
import csc249
csc249.make_submission()